# Batch Process Source HEC-RAS Models
Generate FIM Libraries for multiple HEC-RAS Models

In [1]:
%load_ext autoreload
%autoreload 2

from scripts import *
from scripts.setup import Database
from scripts.setup import CollectionData
from scripts.setup import STACImporter # Remove when get working models is refactored
from scripts.process import *
from scripts.process import JobClient
from scripts.process import ConflateModelBatchProcessor
from scripts.process import extract_submodel_batchProcessor, ReachData
# from time import sleep
# sleep(60*18)

### Instantiate CollectionData and Database

In [ ]:
# All paths should be defined from CollectionData object
collection = CollectionData("") # Insert Collection name - example: "ble_12100302_Medina"

database = Database(collection)

### Get Working Models

In [ ]:
#Instead of polling the STAC Collection as done previously, we gather models_data from what is currently on disk. 
model_ids = collection.get_models()

### Run `conflate_model`

In [ ]:
jobclient = JobClient(collection)

conflate_model_batch = ConflateModelBatchProcessor(collection, jobclient, database)

conflate_model_batch.conflate_model_batch_process()

if conflate_model_batch.stop_on_error and (len(conflate_model_batch.failed) + len(conflate_model_batch.not_accepted) + len(conflate_model_batch.unknown)) > 0:
    raise Exception("One or more model failed. Stopping Execution. Please address and then run below cells.")
# from scripts.qc import poll_and_update_job_status, get_reach_status_by_process
# poll_and_update_job_status(db_path, "conflate_model", "models")
# succeded_models, failed_models, not_accepted_models = get_reach_status_by_process(db_path, "conflate_model", "models")

### Load Conflation Information to Database

In [ ]:
#TODO 
# Create a @dataclass for model_job_status & reach_job_status
succeeded_and_unknown_status_models = [model_id[0] for model_id in conflate_model_batch.succeded + conflate_model_batch.unknown]
load_conflation(succeeded_and_unknown_status_models, database)

### Update `network_to_id` Table in Database
Build a modified network by skipping over reaches eclipsed by adjacent reaches

In [ ]:
update_network(database)

### Get Working Reaches

In [ ]:
reaches = database.get_reaches_by_models()
reach_data = [(data[0], data[2]) for data in reaches]
# reach_data = [ReachData(reach[0], reach[1], reach[2]) for reach in reaches]

### Run `extract_submodel`

In [ ]:
extract_submodel_batch = ExecuteReachStepBatchProcessor(collection, reach_data, jobclient, database)

extract_submodel_batch.execute_extract_submodel_process()

extract_submodel_batch._set_succesful_and_unknown_reaches_list()

if extract_submodel_batch.stop_on_error and (len(extract_submodel_batch.failed) + len(extract_submodel_batch.not_accepted) + len(extract_submodel_batch.unknown)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

# from scripts.qc import poll_and_update_job_status
# poll_and_update_job_status(db_path, "extract_submodel")
# succeded_models, failed_models, not_accepted_models = get_reach_status_by_process(db_path, "extract_submodel")

### Run `create_ras_terrain`

In [ ]:
create_ras_terrain_batch = ExecuteReachStepBatchProcessor(collection, reach_data, jobclient, database) 

create_ras_terrain_batch.execute_process("create_ras_terrain")

if create_ras_terrain_batch.stop_on_error and (len(create_ras_terrain_batch.failed) + len(create_ras_terrain_batch.not_accepted)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")


### Run `create_model_run_normal_depth`

In [ ]:
create_model_run_normal_depth_batch = ExecuteReachStepBatchProcessor(collection, reach_data, jobclient, database) 

create_ras_terrain_batch.execute_process("create_model_run_normal_depth")

if create_ras_terrain_batch.stop_on_error and (len(create_ras_terrain_batch.failed) + len(create_ras_terrain_batch.not_accepted) + len(create_ras_terrain_batch.unknown)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `run_incremental_normal_depth`

In [ ]:
run_incremental_normal_depth_batch = ExecuteReachStepBatchProcessor(collection, reach_data, jobclient, database) 

run_incremental_normal_depth_batch.execute_process("run_incremental_normal_depth")

if run_incremental_normal_depth_batch.stop_on_error and (len(run_incremental_normal_depth_batch.failed) + len(run_incremental_normal_depth_batch.not_accepted) + len(run_incremental_normal_depth_batch.unknown)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `run_known_wse` for Network

In [ ]:
outlet_reaches = [data[0] for data in reaches if data[1] is None]

execute_kwse_for_network([(reach, None) for reach in outlet_reaches], collection, database, jobclient)

In [ ]:
# # Optional if KWSE is not performed
# create_fim_lib_batch = ExecuteReachStepBatchProcessor(collection, reach_data, jobclient, database) 

# create_fim_lib_batch.execute_process("run_incremental_normal_depth")

# if create_fim_lib_batch.stop_on_error and (len(create_fim_lib_batch.failed) + len(create_fim_lib_batch.not_accepted) + len(create_fim_lib_batch.unknown)) > 0:
#     raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

# outlet_reaches = [data[0] for data in reaches if data[1] is None]

## Merge Rating Curves

In [ ]:
load_all_rating_curves(collection.library_dir, database)

## Create Flows2FIM Start Reaches File

In [18]:
from scripts.process import create_f2f_start_file
create_f2f_start_file(outlet_reaches, collection.f2f_start_file)

In [16]:
# !jupyter nbconvert --to html "process copy 2.ipynb"
